<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/gpt_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# 连接到 Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
with open("drive/MyDrive/GPT/惊悚乐园.txt", 'r', encoding='utf-8') as f:
  text=f.read()
chars=sorted(list(set(text)))
print(len(chars))

vocab_size=len(chars)

4815
CPU times: user 690 ms, sys: 54.1 ms, total: 744 ms
Wall time: 2.16 s


In [ ]:
%%time
string_to_int={ ch:i for i,ch in enumerate(chars) }
int_to_string={ i:ch for i,ch in enumerate(chars) }
encode=lambda s: [string_to_int[c] for c in s]
decode=lambda l: ''.join([int_to_string[i] for i in l])

print(encode("封不觉"))

[1291, 184, 3868]
CPU times: user 3.79 ms, sys: 0 ns, total: 3.79 ms
Wall time: 4.15 ms


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size=128
batch_size=256
max_iters=2000
learning_rate=3e-4
eval_iters=100
dropout=0.2
n_embd=496
n_layer=4
n_head=4

data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

cpu
tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181,
        1100,  200, 3868,    0,  477, 1263, 3175,  281, 4787,    0,    1,    1,
           1,    1, 2283, 4186, 2128,  548, 1621, 1606,  225,  986,  159, 4191,
         184,  277, 2043, 2515, 1689, 4784,  235, 2043, 1813, 1694,  793, 3927,
        2616,  159, 1579, 1631, 2043,  272, 3203, 2902, 2104, 3453, 4784, 1233,
         354,  272, 4142, 1486, 4784, 1621, 1660, 4784,  283, 3379, 2707,  183,
        4383, 3947,  159, 4310, 4331,  706,  294, 1293, 2515, 1689, 3377, 2299,
        3833, 4059, 2128, 4784,  333, 2065, 1665,  793,  595, 2342, 2043,  239,
         184,  548, 2902,  159])


In [ ]:
n=int(0.8*len(data))
training_set=data[:n]
test_set=data[n:]

def get_batch(split):
  data=training_set if split=="train" else test_set
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

x,y=get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[1819, 4059, 2128,  ...,    1,    1,  103],
        [ 105,  701, 3860,  ...,  213, 3262,  302],
        [3694, 4341,  102,  ...,  176, 2902,  793],
        ...,
        [3136,  433, 3995,  ..., 2902, 1249,  577],
        [   1,    1,    1,  ..., 4293, 4784, 3412],
        [   1,    1,    1,  ..., 2902, 4747, 2072]])
target:
tensor([[4059, 2128, 2902,  ...,    1,  103, 4191],
        [ 701, 3860, 1706,  ..., 3262,  302,  582],
        [4341,  102,  735,  ..., 2902,  793, 1302],
        ...,
        [ 433, 3995, 3473,  ..., 1249,  577, 4273],
        [   1,    1,    1,  ..., 4784, 3412, 1211],
        [   1,    1, 4487,  ..., 4747, 2072,  680]])


In [ ]:
x=training_set[:block_size]
y=training_set[1:block_size+1]

for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print("when t is:", context,"target is:",t)

when t is: tensor([0]) target is: 0
when t is: tensor([  0, 161]) target is: 1
when t is: tensor([   0,  161, 1621]) target is: 2
when t is: tensor([   0,  161, 1621, 1606]) target is: 3
when t is: tensor([   0,  161, 1621, 1606,  225]) target is: 4
when t is: tensor([   0,  161, 1621, 1606,  225,  986]) target is: 5
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162]) target is: 6
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0]) target is: 7
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343]) target is: 8
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377]) target is: 9
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787]) target is: 10
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181]) target is: 11
when t is: tensor([   0,  161, 1621, 1606,  225,  986,  162,    0,  343, 3377, 4787,  181,
        1100]) target is: 12
when t is: tensor([

In [ ]:
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()
    self.key=nn.Linear(n_embd,head_size,bias=False)
    self.query=nn.Linear(n_embd,head_size,bias=False)
    self.value=nn.Linear(n_embd,head_size,bias=False)
    self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    # input:(batch,time-step,channels)
    # output:(batch,time-step,head size)
    B,T,C=x.shape
    k=self.key(x) # (B,T,hs)
    q=self.query(x) # (B,T,hs)
    wei=q@k.transpose(-2,-1)*k.shape[-1]**-0.5 #(B,T,hs)@(B,hs,T)->(B,T,T)

    wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) # (B,T,T)
    wei=F.softmax(wei,dim=-1) # (B,T,T)
    wei=self.dropout(wei)
    v=self.value(x) #(B,T,hs)
    out=wei@v #(B,T,T)@(B,T,hs)->(B,T,hs)
    return out

class MultiHeadAttention(nn.Module):
  def __init__(self,num_heads,head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(head_size*num_heads,n_embd)
    self.dropout=nn.Dropout(dropout)

  def forward(self,x):
    out=torch.cat([h(x) for h in self.heads],dim=-1) # (B,T,[h1*4,h2*4...])
    out=self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  def __init__(self,n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd,4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd,n_embd),
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)

class Block(nn.Module):
  def __init__(self,n_embd,n_head):
    super().__init__()
    head_size=n_embd//n_head
    self.sa=MultiHeadAttention(n_head,head_size)
    self.ffwd=FeedForward(n_embd)
    self.ln1=nn.LayerNorm(n_embd)
    self.ln2=nn.LayerNorm(n_embd)

  def forward(self,x):
    y=self.sa(x)
    x=self.ln1(x+y)
    y=self.ffwd(x)
    x=self.ln2(x+y)
    return x

class GPTLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table=nn.Embedding(block_size,n_embd)
    self.blocks=nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
    self.ln_f=nn.LayerNorm(n_embd) # final lauer norm(help converge better)
    self.lm_head=nn.Linear(n_embd,vocab_size)
    self.apply(self._init_weights)

  def _init_weights(self,module):
    if isinstance(module,nn.Linear):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module,nn.Embedding):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

  def forward(self,index,targets=None):
    batch,time=index.shape

    # print(f"Time: {time}")
    tok_emb=self.token_embedding_table(index) # (B,T,C)
    pos_emb=self.position_embedding_table(torch.arange(time,device=device)) # (T,C)
    x=tok_emb+pos_emb # (B,T,C)
    x=self.blocks(x) # (B,T,C)
    x=self.ln_f(x) # (B,T,C)
    logits=self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss=None
    else:
      batch,time,channel=logits.shape
      logits=logits.view(batch*time,channel)
      targets=targets.view(batch*time)
      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,index,max_new_tokens):
    # index is (batch,channel)
    for _ in range(max_new_tokens):
      logits,loss=self.forward(index)
      logits=logits[:,-1,:] # become batch channel
      probs=F.softmax(logits,dim=-1) # get the probabilities
      index_next=torch.multinomial(probs,num_samples=1)
      index=torch.cat((index,index_next),dim=1)
    return index

model=GPTLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=100)[0].tolist())
print(generated_chars)


姥缕拆壕ａ伛励ｓ湛于噌G陲胶寸毕皱宫影协巴趁岛嫖登绷额唱哑泣君裆蛀虢鸡萦咎嫣卉镯票，頂渲眶忖淡愧化益拘袭褒语查人恬%丙译谢赌豁街砂扣弃ｏ珊驮萃屈搀犄烜抄闹裾难咫骨祗褚0吨蛟否屿疏扁皓袈^凡支掖五敞覆酿


In [ ]:
@torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ['train','val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y=get_batch(split)
      logits,loss=model(X,Y)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
  xb,yb=get_batch('train')
  logits,loss=model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter%eval_iters==0:
    losses=estimate_loss()
    print(f"step:{iter},train loss:{losses['train']:.4f},val loss:{losses['val']:.4f}")
print(loss.item())

step:0,train loss:7.9211,val loss:7.9085
step:100,train loss:4.2441,val loss:4.3704
step:200,train loss:3.8579,val loss:4.0234
step:300,train loss:3.6438,val loss:3.8353
step:400,train loss:3.4794,val loss:3.7017
step:500,train loss:3.3550,val loss:3.6119
step:600,train loss:3.2467,val loss:3.5159
step:700,train loss:3.1543,val loss:3.4562
step:800,train loss:3.0859,val loss:3.3985
step:900,train loss:3.0217,val loss:3.3587
step:1000,train loss:2.9644,val loss:3.3267
step:1100,train loss:2.9142,val loss:3.2945
step:1200,train loss:2.8634,val loss:3.2796
step:1300,train loss:2.8205,val loss:3.2475
step:1400,train loss:2.7808,val loss:3.2419
step:1500,train loss:2.7437,val loss:3.2249
step:1600,train loss:2.7102,val loss:3.2097
step:1700,train loss:2.6772,val loss:3.1976
step:1800,train loss:2.6463,val loss:3.1819
step:1900,train loss:2.6212,val loss:3.1848
2.7408809661865234


In [ ]:
torch.cuda.set_device(0)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=decode(m.generate(context,max_new_tokens=120)[0].tolist())
print(generated_chars)


    “【115132】【86250公斤】
    此处走势十分钟，觉哥便唤出五个手电筒的电脑，开始，来自这边……通常情况下，他自然早就没遇到了什么屏蔽啊。
    三，在十五分钟前，来到非得住气，并排杀了名和客人。
    “于是，他以


In [ ]:
torch.save(m.state_dict(), 'model.pth')

In [ ]:
model1 = GPTLanguageModel(vocab_size)
model1.load_state_dict(torch.load('drive/MyDrive/GPT/model-1.pth', map_location=torch.device('cpu')))#
model1.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(4815, 496)
  (position_embedding_table): Embedding(128, 496)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=496, out_features=124, bias=False)
            (query): Linear(in_features=496, out_features=124, bias=False)
            (value): Linear(in_features=496, out_features=124, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=496, out_features=496, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=496, out_features=1984, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1984, out_features=496, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((496,), eps=1e-05, elementwise_a

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torch.cuda.set_device(0)
model1.to(device)
max_tokens_per_generation = 120
total_tokens_to_generate = 400  # 需要生成的总词数
generated_text = ""


while len(generated_text) < total_tokens_to_generate:
    remaining_tokens = total_tokens_to_generate - len(generated_text.split())
    tokens_to_generate = min(max_tokens_per_generation, remaining_tokens)

    if generated_text:
      context = torch.tensor([[tokenizer.encode(generated_text)[-1]]], dtype=torch.long, device=device)
    else:
      default_pad_token_id = 0
      context = torch.tensor([[default_pad_token_id]], dtype=torch.long, device=device)

    generated_chars = decode(model1.generate(context, max_new_tokens=tokens_to_generate)[0].tolist())

    # print(generated_chars)

    generated_text += generated_chars

print(generated_text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


    …………
    “年轻人还自解事儿呢。”尤先生拉长了嗓门儿道：“老子同学，你是说了几年就挺嫌麻烦的嘛。”
    “非一部长吓人，而这么合适。”封不觉耸肩嘀咕道，“姓几乎都只有你一个了。”

第0854章 笑濒临终的焦点。
   之后的三名摄像师伴随着操作杆的一句。
    “喂！天哪！奏感！”此时，刀脉搏命的双倍后，他立刻反而沉……“你们也每次都会曾听过这事儿”朝前挪一步的小叹示意摇。
    “唉……否则咱们就该不会爬到了。”封不觉双手插袋似的手腕，轻松地说道。
丞时候多吉诃德破坏过程的事情，还是这样嘲讽……‘改变了’……”
    “你的命令，是有名气种人而已。”奥因克接着说道，“我主要是在二十四了打起来，让蝙蝠女王的弟子逼我们进入了今天，我就回到了任务栏。”他将指示附了房间的封不觉：“另外，你是怎习以为从这些人物类出头去练级战斗直接传达。
    这种补偿，想必有眼下有所推定得具有勾结，也没了结束的结果。
    而这次已经结束了，林颜的那次再想回来了，很快便了身体。接下来只需过一次，这个过程便可佐证她的看穿越多高一些悬言，也已经把命
